## Classification with Logistic Regression on Mouse Ketamine Dataset

In [6]:
# packages
import os
import pandas as pd
import math
from scipy import io
import numpy as np
from numpy import squeeze
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import zero_one_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot') 

### Load in data

In [2]:
allData = pd.read_csv('sessionTrialTable.csv')

In [3]:
allData.keys()

Index(['animalName', 'sessionDate', 'trialNum', 'totalCellNum', 'gender',
       'genotype', 'weight_g', 'ketamine_day', 'correlationScore',
       'lickAccuracy', 'lickNumber', 'avgFR', 'avgSingleCellVariance',
       'varianceFR', 'avgTrialSpeed', 'varianceSpeed', 'medianCellDepth',
       'timeSinceKetamine', 'ketamineAdministered'],
      dtype='object')

In [4]:
# Check size information
print("num_cols =",len(allData.keys()))
print("num_rows =",len(allData))

# Check for duplicate rows
print("num_dup =",np.sum(pd.DataFrame.duplicated(allData)))

num_cols = 19
num_rows = 5000
num_dup = 0


In [5]:
# Check for NaNs and see where they are coming from
np.sum(pd.isna(allData))

animalName               0
sessionDate              0
trialNum                 0
totalCellNum             0
gender                   0
genotype                 0
weight_g                 0
ketamine_day             0
correlationScore         0
lickAccuracy             0
lickNumber               0
avgFR                    1
avgSingleCellVariance    1
varianceFR               3
avgTrialSpeed            0
varianceSpeed            0
medianCellDepth          0
timeSinceKetamine        0
ketamineAdministered     0
dtype: int64

In [6]:
# Simply drop any rows with nans since we have so few
allDataNN = pd.DataFrame.dropna(allData,'index')
print("After Drop NaN")
print("num_rows =",len(allDataNN))

After Drop NaN
num_rows = 4997


### Logistic Regression Classifier

In [7]:
ketBool = allDataNN['ketamineAdministered']
timeSinceKetamine = allDataNN['timeSinceKetamine']
sessionDate = allDataNN['sessionDate']
trialNum = allDataNN['trialNum']
neuralData = allDataNN[['animalName', 'totalCellNum',
       'gender', 'genotype', 'weight_g',
       'ketamine_day', 'correlationScore', 'lickAccuracy',
       'lickNumber', 'avgFR', 'avgSingleCellVariance',
       'varianceFR', 'avgTrialSpeed', 'varianceSpeed',
       'medianCellDepth']]

In [8]:
# Convert categorical columns
le = LabelEncoder()
neuralData_LE = neuralData.copy()
neuralData_LE['animalName'] = le.fit_transform(neuralData_LE['animalName'])
neuralData_LE['gender'] = le.fit_transform(neuralData_LE['gender'])
neuralData_LE['genotype'] = le.fit_transform(neuralData_LE['genotype'])

In [9]:
X, X_test, y, y_test = train_test_split(neuralData_LE,ketBool.values.ravel(), test_size=0.2)
# Now we save X_test and y_test for next part of the project!

In [10]:
# Split for cross validation, use 10 folds
num_folds = 10
XA = np.array(X)
yA = np.array(y)
X_train = []
X_test = []
y_train = []
y_test = []
skf = StratifiedKFold(n_splits=num_folds)
for train_index, test_index in skf.split(XA, yA):
    X_train.append(XA[train_index])
    X_test.append(XA[test_index])
    y_train.append(yA[train_index])
    y_test.append(yA[test_index])

In [11]:
# Run basic log reg model on train set, check performance against train
# Run model, tuning over C_param (l2 penalty by default)
for C_param in [0.01, 0.1, 1, 10, 100, 1000, 5000, 10000, 50000]:
    model = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000,C = C_param).fit(X, y) 
    y_pred = model.predict(X)
    print(zero_one_loss(y, y_pred))
# Let's use C = 100

0.1848886664998749
0.16562421816362272
0.15186389792344257
0.14836127095321494
0.14535901926444839
0.14786089567175387
0.14961220915686768
0.14660995746810113
0.14660995746810113


Best loss was at C=10000, so let's use that! Note that the resulting loss score was 0.1411.

In [12]:
C_param = 10000
zo_loss = []
accuracy = []
num_folds=10
for i in range(0,num_folds):
    LRmodel = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000, C = C_param).fit(X_train[i], y_train[i]) 
    y_pred = model.predict(X_test[i])
    zo_loss.append(zero_one_loss(y_test[i],y_pred))
    accuracy.append(accuracy_score(y_test[i],y_pred))

avg_zo_loss = np.mean(zo_loss)
avg_acc = np.mean(accuracy)
print("Average zero-one loss across folds:",avg_zo_loss)
print("Average accuracy across folds:",avg_acc)


Average zero-one loss across folds: 0.14660081313008205
Average accuracy across folds: 0.8533991868699179


Average zero-one loss across 10-fold CV was 0.1466, only very slightly worse than our overall training error.

#### Now try with standardization

In [13]:
stdNeuralData = StandardScaler().fit_transform(neuralData_LE) 
X, X_test, y, y_test = train_test_split(stdNeuralData,ketBool.values.ravel(), test_size=0.2)

/home/browne/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/browne/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [14]:
# Split for cross validation, use 10 folds
num_folds = 10
XA = np.array(X)
yA = np.array(y)
X_train = []
X_test = []
y_train = []
y_test = []
skf = StratifiedKFold(n_splits=num_folds)
for train_index, test_index in skf.split(XA, yA):
    X_train.append(XA[train_index])
    X_test.append(XA[test_index])
    y_train.append(yA[train_index])
    y_test.append(yA[test_index])

In [15]:
# Run basic log reg model on train set, check performance against train
# Run model, tuning over C_param (l2 penalty by default)
for C_param in [0.01, 0.1, 1, 10, 100, 1000]:
    model = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000,C = C_param).fit(X, y) 
    y_pred = model.predict(X)
    print(zero_one_loss(y, y_pred))

0.1553665248936702
0.15061295971978983
0.14836127095321494
0.14786089567175387
0.14811108331248435
0.14811108331248435


Best loss was at C=10, so let's use that! Note that the resulting loss score was 0.1471.

In [16]:
C_param = 10
zo_loss = []
accuracy = []
num_folds=10
for i in range(0,num_folds):
    LRmodel = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000, C = C_param).fit(X_train[i], y_train[i]) 
    y_pred = model.predict(X_test[i])
    zo_loss.append(zero_one_loss(y_test[i],y_pred))
    accuracy.append(accuracy_score(y_test[i],y_pred))

avg_zo_loss = np.mean(zo_loss)
avg_acc = np.mean(accuracy)
print("Average zero-one loss across folds:",avg_zo_loss)
print("Average accuracy across folds:",avg_acc)


Average zero-one loss across folds: 0.1481146616541353
Average accuracy across folds: 0.8518853383458647


Average zero-one loss across 10-fold CV was 0.1471, the same within significant digits as our overall training error.

In [17]:
# TEST SET CHECKS - FOR LATER
# Make predictions# Let's use C = 100
#y_pred = model.predict(X_test)
# Keep prediction probability, not using for now, consider adding a threshold
#y_pred_proba = model.predict_proba(X_test)
#print(zero_one_loss(y_test, y_pred))

#### Now with augmentation!

In [18]:
AugData = neuralData_LE.copy()

In [19]:
AugData['animalNamexCorrelationScore'] = AugData['animalName']*AugData['correlationScore']
AugData['animalNamexLickAccuracy'] = AugData['animalName']*AugData['lickAccuracy']
AugData['animalNamexLickNumber'] = AugData['animalName']*AugData['lickNumber']
AugData['animalNamexAvgFR'] = AugData['animalName']*AugData['avgFR']
AugData['animalNamexAvgSingleCellVariance'] = AugData['animalName']*AugData['avgSingleCellVariance']
AugData['animalNamexVarianceFR'] = AugData['animalName']*AugData['varianceFR']
AugData['animalNamexAvgTrialSpeed'] = AugData['animalName']*AugData['avgTrialSpeed']
AugData['animalNamexVarianceSpeed'] = AugData['animalName']*AugData['varianceSpeed']

AugData['totalCellNumxCorrelationScore'] = AugData['totalCellNum']*AugData['correlationScore']
AugData['totalCellNumxLickAccuracy'] = AugData['totalCellNum']*AugData['lickAccuracy']
AugData['totalCellNumxLickNumber'] = AugData['totalCellNum']*AugData['lickNumber']
AugData['totalCellNumxAvgFR'] = AugData['totalCellNum']*AugData['avgFR']
AugData['totalCellNumxAvgSingleCellVariance'] = AugData['totalCellNum']*AugData['avgSingleCellVariance']
AugData['totalCellNumxVarianceFR'] = AugData['totalCellNum']*AugData['varianceFR']
AugData['totalCellNumxAvgTrialSpeed'] = AugData['totalCellNum']*AugData['avgTrialSpeed']
AugData['totalCellNumxVarianceSpeed'] = AugData['totalCellNum']*AugData['varianceSpeed']

AugData['genderxCorrelationScore'] = AugData['gender']*AugData['correlationScore']
AugData['genderxLickAccuracy'] = AugData['gender']*AugData['lickAccuracy']
AugData['genderxLickNumber'] = AugData['gender']*AugData['lickNumber']
AugData['genderxAvgFR'] = AugData['gender']*AugData['avgFR']
AugData['genderxAvgSingleCellVariance'] = AugData['gender']*AugData['avgSingleCellVariance']
AugData['genderxVarianceFR'] = AugData['gender']*AugData['varianceFR']
AugData['genderxAvgTrialSpeed'] = AugData['gender']*AugData['avgTrialSpeed']
AugData['genderxVarianceSpeed'] = AugData['gender']*AugData['varianceSpeed']

AugData['genotypexCorrelationScore'] = AugData['genotype']*AugData['correlationScore']
AugData['genotypexLickAccuracy'] = AugData['genotype']*AugData['lickAccuracy']
AugData['genotypexLickNumber'] = AugData['genotype']*AugData['lickNumber']
AugData['genotypexAvgFR'] = AugData['genotype']*AugData['avgFR']
AugData['genotypexAvgSingleCellVariance'] = AugData['genotype']*AugData['avgSingleCellVariance']
AugData['genotypexVarianceFR'] = AugData['genotype']*AugData['varianceFR']
AugData['genotypexAvgTrialSpeed'] = AugData['genotype']*AugData['avgTrialSpeed']
AugData['genotypexVarianceSpeed'] = AugData['genotype']*AugData['varianceSpeed']

AugData['weight_gxCorrelationScore'] = AugData['weight_g']*AugData['correlationScore']
AugData['weight_gxLickAccuracy'] = AugData['weight_g']*AugData['lickAccuracy']
AugData['weight_gxLickNumber'] = AugData['weight_g']*AugData['lickNumber']
AugData['weight_gxAvgFR'] = AugData['weight_g']*AugData['avgFR']
AugData['weight_gxAvgSingleCellVariance'] = AugData['weight_g']*AugData['avgSingleCellVariance']
AugData['weight_gxVarianceFR'] = AugData['weight_g']*AugData['varianceFR']
AugData['weight_gxAvgTrialSpeed'] = AugData['weight_g']*AugData['avgTrialSpeed']
AugData['weight_gxVarianceSpeed'] = AugData['weight_g']*AugData['varianceSpeed']

AugData['ketamine_dayxCorrelationScore'] = AugData['ketamine_day']*AugData['correlationScore']
AugData['ketamine_dayxLickAccuracy'] = AugData['ketamine_day']*AugData['lickAccuracy']
AugData['ketamine_dayxLickNumber'] = AugData['ketamine_day']*AugData['lickNumber']
AugData['ketamine_dayxAvgFR'] = AugData['ketamine_day']*AugData['avgFR']
AugData['ketamine_dayxAvgSingleCellVariance'] = AugData['ketamine_day']*AugData['avgSingleCellVariance']
AugData['ketamine_dayxVarianceFR'] = AugData['ketamine_day']*AugData['varianceFR']
AugData['ketamine_dayxAvgTrialSpeed'] = AugData['ketamine_day']*AugData['avgTrialSpeed']
AugData['ketamine_dayxVarianceSpeed'] = AugData['ketamine_day']*AugData['varianceSpeed']

AugData['medianCellDepthxCorrelationScore'] = AugData['medianCellDepth']*AugData['correlationScore']
AugData['medianCellDepthxLickAccuracy'] = AugData['medianCellDepth']*AugData['lickAccuracy']
AugData['medianCellDepthxLickNumber'] = AugData['medianCellDepth']*AugData['lickNumber']
AugData['medianCellDepthxAvgFR'] = AugData['medianCellDepth']*AugData['avgFR']
AugData['medianCellDepthxAvgSingleCellVariance'] = AugData['medianCellDepth']*AugData['avgSingleCellVariance']
AugData['medianCellDepthxVarianceFR'] = AugData['medianCellDepth']*AugData['varianceFR']
AugData['medianCellDepthxAvgTrialSpeed'] = AugData['medianCellDepth']*AugData['avgTrialSpeed']
AugData['medianCellDepthxVarianceSpeed'] = AugData['medianCellDepth']*AugData['varianceSpeed']

In [20]:
# Standardize data
stdNeuralDataAug = StandardScaler().fit_transform(AugData) 

/home/browne/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/browne/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [21]:
X, X_test, y, y_test = train_test_split(stdNeuralDataAug,ketBool.values.ravel(), test_size=0.2)

In [22]:
# Split for cross validation, use 10 folds
num_folds = 10
XA = np.array(X)
yA = np.array(y)
X_train = []
X_test = []
y_train = []
y_test = []
skf = StratifiedKFold(n_splits=num_folds)
for train_index, test_index in skf.split(XA, yA):
    X_train.append(XA[train_index])
    X_test.append(XA[test_index])
    y_train.append(yA[train_index])
    y_test.append(yA[test_index])

In [24]:
# Run basic log reg model on train set, check performance against train
# Run model, tuning over C_param (l2 penalty by default)
for C_param in [0.01, 0.1, 1, 10, 100, 1000, 10000]:
    model = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000,C = C_param).fit(X, y) 
    y_pred = model.predict(X)
    print(zero_one_loss(y, y_pred))

0.12109081811358524
0.09582186639979984
0.08006004503377528
0.07305479109332003
0.07305479109332003
0.07230422817112836
0.07205404053039777


In [25]:
C_param = 1000
zo_loss = []
accuracy = []
num_folds=10
for i in range(0,num_folds):
    LRmodel = linear_model.LogisticRegression(solver='lbfgs',max_iter=10000, C = C_param).fit(X_train[i], y_train[i]) 
    y_pred = model.predict(X_test[i])
    zo_loss.append(zero_one_loss(y_test[i],y_pred))
    accuracy.append(accuracy_score(y_test[i],y_pred))

avg_zo_loss = np.mean(zo_loss)
avg_acc = np.mean(accuracy)
print("Average zero-one loss across folds:",avg_zo_loss)
print("Average accuracy across folds:",avg_acc)


Average zero-one loss across folds: 0.07204581176559488
Average accuracy across folds: 0.927954188234405


In [26]:
model = linear_model.LogisticRegression(solver='lbfgs',max_iter=1000,C = 10000).fit(X, y) 
y_pred = model.predict(X)

In [27]:
zero_one_loss(y,y_pred)

0.07205404053039777

In [28]:
accuracy_score(y,y_pred)

0.9279459594696022

In [37]:
cm = confusion_matrix(y,y_pred)

In [40]:
tn = cm[0,0]
fn = cm[1,0]
tp = cm[1,1]
fp = cm[0,1]

In [41]:
tn

1832

In [42]:
fn

152

In [43]:
tp

1877

In [44]:
fp

136

In [47]:
cm

array([[1832,  136],
       [ 152, 1877]])

In [50]:
1832+152+136+1877

3997